In [38]:
%%capture
!pip install tiktoken

In [39]:
import torch
from torch.utils.data import Dataset, DataLoader    
import tiktoken

- Ta sẽ sử dụng `DataLoader` được viết ở phần `3. Data_sampling.ipynb`

In [40]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        # max_length: length of each row
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader_v1(txt, max_length=256, stride=128, batch_size=4, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

    return dataloader

In [41]:
with open("./the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

- Bây giờ, ta sẽ xét đến kích thước embedding thực tế hơn (`256`). Tuy số chiều này nhỏ hơn nhiều so với GPT-3 (`12,288`), nhưng hợp lí cho thực nghiệm

In [42]:
output_dim = 256
vocab_size = 50257  # BPE vocabulary size
torch.manual_seed(123)
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


- Token ID tensor có số chiều là `8x4`, có nghĩa là mỗi `batch` sẽ có 8 mẫu với 4 tokens mỗi mẫu.

- Giờ chúng ta sẽ tiến hành `embed` qua `embedding_layer`

In [43]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


- Số chiều `8x4x256` nghĩa là "8 mẫu với 4 tokens mỗi mẫu, mỗi token được biểu diễn dưới dạng `vector 256 chiều`".

- Dưới đây là số chiều của `sample thứ 1`.

In [44]:
# First sample
print(token_embeddings[0])

print(token_embeddings[0].shape)

tensor([[-0.0640,  0.3317,  0.1070,  ...,  0.5349, -0.8024, -2.3238],
        [-0.3525,  0.3509,  0.9873,  ..., -1.8466, -1.7034,  0.3223],
        [ 1.0017,  0.9299, -1.2633,  ..., -1.2256,  1.1179,  0.1343],
        [ 0.7996,  2.2837, -0.6525,  ..., -1.1217,  0.4706,  0.1531]],
       grad_fn=<SelectBackward0>)
torch.Size([4, 256])


In [45]:
context_length = max_length
torch.manual_seed(123)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)


torch.Size([4, 256])


- `positional embedding` được khởi tạo với số chiều giống với số chiều của mỗi `sample`.

In [46]:
pos_embeddings

tensor([[ 0.3374, -0.1778, -0.3035,  ...,  1.3337,  0.0771, -0.0522],
        [ 0.2386,  0.1411, -1.3354,  ..., -0.0315, -1.0640,  0.9417],
        [-1.3152, -0.0677, -0.1350,  ..., -0.3181, -1.3936,  0.5226],
        [ 0.2579,  0.3420, -0.8168,  ..., -0.4840, -0.2713, -0.0774]],
       grad_fn=<EmbeddingBackward0>)

- Giờ ta sẽ `add` từng `positional embedding` 4x256 chiều với từng `token embedding` 4x256 chiều trong *mỗi batches* gồm 8 token embedding.

In [47]:
# pos_embeddings -> [4, 256] -> [1, 4, 256] -> multiple by 8 batches -> [8, 4, 256]
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
